In [26]:
# Import necessary libraries
import requests
import json

# Define the FastAPI endpoints
PREDICT_ENDPOINT = "http://127.0.0.1:8000/predict"
STORE_TWEET_ENDPOINT = "http://127.0.0.1:8000/store_tweet"

In [27]:
# Function to predict if a tweet is hate speech
def predict_tweet(text, user):
    response = requests.post(PREDICT_ENDPOINT, json={"text": text, "user": user})
    if response.status_code == 200:
        return response.json()
    else:
        print("Prediction failed:", response.json())
        return None


In [28]:
# Function to store a tweet if it's safe to post
def store_tweet(tweet_id, user_id, retweet_id=None):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id
    }
    response = requests.post(STORE_TWEET_ENDPOINT, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        print("Storing tweet failed:", response.json())
        return None

In [29]:
# Example tweet text and user
tweet_text = "This is an example tweet for testing purposes."
user_id = "example_user"

In [17]:
#Predict if the tweet is hate speech
prediction_result = predict_tweet(tweet_text, user_id)

if prediction_result:
    print("Prediction Result:", prediction_result)
    
    # Check if the tweet is safe to post (logreg_result should be 0)
    if prediction_result["logreg_result"] == 0:
        tweet_id = prediction_result["tweet_id"]
        
        # Step 3: Store the tweet in the posted tweets database
        store_result = store_tweet(tweet_id, user_id)
        if store_result:
            print("Tweet successfully posted:", store_result)
    else:
        print("Tweet is considered hate speech and will not be posted.")
else:
    print("Prediction failed.")

Prediction Result: {'tweet_id': '181512ed', 'tweet': 'This is an example tweet for testing purposes.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.01582927467824479, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None}
Tweet successfully posted: {'tweet_id': '181512ed', 'retweet_id': None, 'text': 'This is an example tweet for testing purposes.', 'retweets': 0, 'id': 1, 'user_id': 'example_user', 'likes': 0, 'safety_status': None}


## create random users, and post tweets under test.csv under their names

In [30]:
import pandas as pd
import random
import requests
from datetime import datetime, timezone

base_url = "http://localhost:8000"

# Define users
users = [f"user{i}" for i in range(1, 9)]

In [48]:
# Read tweets from CSV
test_csv_path = '../data/test.csv'
df = pd.read_csv(test_csv_path)

# Shuffle tweets and split into 8 equal parts
tweets = df['tweets'].tolist()
random.shuffle(tweets)
tweets = tweets[40:43]
num_tweets_per_user = len(tweets) // len(users)

In [49]:
num_tweets_per_user

1

In [50]:
# Function to post a tweet for prediction
def post_tweet_for_prediction(tweet, user):
    response = requests.post(f'{base_url}/predict', json={"text": tweet, "user": user})
    return response.json()

In [51]:
def store_posted_tweet(tweet_id, retweet_id, user_id, text, likes, retweets, safety_status, created_at):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id,
        "text": text,
        "likes": likes,
        "retweets": retweets,
        "safety_status": safety_status,
        "created_at": created_at
    }
    response = requests.post(f'{base_url}/store_tweet', json=data)
    return response.json()

In [52]:
# Function to simulate user submissions
def simulate_user_submission():
    for i, user in enumerate(users):
        user_tweets = tweets[i * num_tweets_per_user : (i + 1) * num_tweets_per_user]
        for tweet in user_tweets:
            prediction = post_tweet_for_prediction(tweet, user)
            print("Prediction Result:", prediction)
            
            if prediction["logreg_result"] != 1:
                created_at = datetime.now(timezone.utc).isoformat()  # Convert datetime to ISO format string

                store_response = store_posted_tweet(
                    tweet_id=prediction["tweet_id"],
                    retweet_id=None,
                    user_id=user,
                    text=prediction["tweet"],
                    likes=prediction["likes"],
                    retweets=prediction["retweets"],
                    safety_status=None,  # Initial safety status is None
                    created_at=created_at
                )
                print("Store Tweet Response:", store_response)

In [53]:
# Run the simulation
simulate_user_submission()

Prediction Result: {'tweet_id': 'f9f8aec5', 'tweet': 'ane detest dark men it makes me nauseated', 'user': 'user4', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.990918348358057, 'logreg_result': 1, 'cnn_prob': None, 'cnn_result': None, 'created_at': '2024-07-25T07:47:29.668075'}
Prediction Result: {'tweet_id': 'b401255b', 'tweet': 'npov you would need strong consensus to change a longstanding lead on a core content policy the changes you made removed some fairly important points', 'user': 'user2', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.01816162792497496, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None, 'created_at': '2024-07-25T07:47:31.761220'}
Store Tweet Response: {'id': 19, 'user_id': 'user2', 'likes': 0, 'safety_status': None, 'text': 'npov you would need strong consensus to change a longstanding lead on a core content policy the changes you made removed some fairly important points', 'tweet_id': 'b401255b', 'retweet_id': None, 'retweets': 0, 'created_at': '2024-07-25T

## 3 users to report tweets

In [38]:
tweet_ids = ["606799a0", "c888cb2a", "c888cb2a"]
users = ["user4", "user2", "user6"]

def report_tweet(report_data):
    url = f"{base_url}/report_tweet"
    response = requests.post(url, json=report_data)
    
    if response.status_code == 200:
        print("Report submitted successfully.")
    else:
        print(f"Failed to submit report: {response.status_code}")
        print(f"Response: {response.json()}")

In [39]:
for i in range(3):
    report_data = {
        "tweet_id": tweet_ids[i],
        "user_id": users[i],
        "safety_status": 1,
        "reported_at": datetime.now(timezone.utc).isoformat()
    }
    report_tweet(report_data)

Report submitted successfully.
Report submitted successfully.
Report submitted successfully.
